# Pointage automatique du temps d'arrivé des ondes P et S

In [74]:
import numpy as np

In [75]:
import matplotlib

matplotlib.use('pgf') # figures latex

import matplotlib.pyplot as plt

plt.style.use("bmh")

matplotlib.rcParams.update({
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
    })

In [76]:
import obspy.signal
from obspy import read, read_inventory, UTCDateTime
from obspy.clients.fdsn import Client

In [77]:
from p_s_onset_determination import *

## Charger les données

In [78]:
# # Téléchargement
# Séisme induit de Strasbourg
starttime = UTCDateTime(2021, 6, 26, 3, 0, 0)
duration = 3*60  # secondes
station = "ILLF" # station d'Illkirch
# client = Client("RESIF")  # Réseau français
# S = client.get_waveforms(network="FR", station="ILLF", location="00", channel="HH*", starttime=starttime, endtime=starttime+duration)
# inventory = client.get_stations(network="FR", station=station, channel="HH*", starttime=starttime, level="response")

In [79]:
# Chargement d'un fichier local
S = read(f"data/{station}.miniseed")
inventory = read_inventory(f"data/{station}.xml")

## Traitement des données

In [80]:
pre_process_data(S, inventory)

In [81]:
# Sélection de la trace d'intérêt
T = S.select(channel="HH2")[0]
times = T.times(type='relative')
data = T.data
n = len(data)
#dt = times[1]-times[0]
dt = T.stats.sampling_rate

In [82]:
t1, t2 = 0, 180
plt.plot(times, data, color='grey', linewidth=1.5)
plt.xlabel('Temps [s]')
plt.ylabel('Vitesse de déplacement [m/s]')
plt.xlim([t1, t2])

(0.0, 180.0)

## Conversion du signal en enveloppe

### Valeur absolue du signal

In [83]:
abs_data = np.abs(data)

In [84]:
t1, t2 = 0, 180
plt.plot(times, abs_data, color='red', linewidth=1, label="Valeur absolue")
plt.plot(times, data, color='grey', linewidth=1, label="Signal", alpha=.5)
plt.legend()
plt.xlabel('Temps [s]')
plt.ylabel('Amplitude [m/s]')
plt.xlim([t1, t2])

(0.0, 180.0)

### Carré du signal

In [85]:
sqa_data = data**2

In [86]:
fig, axs = plt.subplots(2, sharex=True)
ax1, ax2 = axs

t1, t2 = times[0], times[-1]

color = 'tab:grey'
ax1.set_ylabel('Amplitude [m/s]', color=color)
ax1.plot(times, data, color=color, linewidth=1, label="Signal")
ax1.set_xlim([t1, t2])
ax1.tick_params(axis='y', labelcolor=color)

color = 'tab:red'
ax2.set_ylabel('Amplitude [m/s]$^2$', color=color)
ax2.plot(times, sqa_data, color=color, linewidth=1, label="Carré")
ax2.set_xlim([t1, t2])
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_xlabel('Temps [s]')

fig.tight_layout()

### Dérivée temporelle

In [87]:
der_data = [0]
for k in range(n-1):
    dy = (data[k+1] - data[k])
    der_data.append(dy/dt)

### Envelope de Stewart

In [88]:
fc_stewart = mdx_stewart(times, data, 10)

In [89]:
fig, axs = plt.subplots(2, sharex=True, figsize=(6, 3))
ax1, ax2 = axs

t1, t2 = times[0], times[-1]
t1, t2 = 40, 100

color = 'tab:grey'
ax1.set_ylabel('Signal')
ax1.plot(times, data, color=color, linewidth=1.5)
ax1.set_xlim([t1, t2])
ax1.tick_params(axis='y')

ax2.set_ylabel('Enveloppe')
ax2.plot(times, np.abs(fc_stewart), color='tab:blue', linewidth=1.5)
ax2.set_xlim([t1, t2])
ax2.set_xlabel('Temps [s]')

fig.tight_layout()
plt.savefig('images/envelope_stewart.pgf', format='pgf')

### Enveloppe supérieure du signal

#### Approximation par méthode géométrique

In [90]:
#env_times, env_data = envelope_approx(times, data)
env_times, env_data = envelope_approx_np(times, data)

In [91]:
t1, t2 = 43, 65
plt.plot(times, data, color='grey', linewidth=1.5, label='Signal')
plt.plot(env_times, env_data, color='red', linewidth=2, linestyle='solid', label='Enveloppe sup. (approx.)')
plt.xlabel('Temps [s]')
plt.ylabel('Amplitude [m/s]')
plt.xlim([t1, t2])
plt.legend()

#### Calcul exact

In [92]:
env_data2 = obspy.signal.filter.envelope(data)

In [93]:
t1, t2 = 44, 53
plt.figure(figsize=(6,3))
plt.plot(times, data, color='grey', linewidth=1.5, label='Signal')
plt.plot(times, abs_data, label='Valeur absolue', color='black', linewidth=1, linestyle='dotted')
plt.plot(env_times, env_data, color='red', linewidth=2.5, alpha=1, linestyle='solid', label='Enveloppe sup. (approx.)')
plt.plot(times, env_data2, color='green', linewidth=2.5, alpha=0.7, linestyle='solid', label='Enveloppe sup.')
plt.xlabel('Temps [s]')
plt.ylabel('Amplitude [m/s]')
plt.xlim([t1, t2])
plt.ylim([-1e-5, 2e-5])
plt.legend()
plt.savefig('images/envelope_hilbert-geom.pgf', format='pgf')


### Enveloppe de Allen

In [94]:
#allen_data = allen_envelope(data)
allen_data = allen_envelope_np(data)

In [95]:
#baer_data = baer_envelope(times, data)
baer_data = baer_envelope_np(times, data)

In [96]:
t1, t2 = 45, 46.5
l = 1.5
fig = plt.figure(figsize=(6, 4))
ax = plt.subplot(111)
plt.plot(times, data, label='Signal', linewidth=5, color='grey')
plt.plot(times, abs_data, label='Valeur absolue', color='black', linewidth=l, linestyle='dotted')
#plt.plot(env_times, env_data, label='Enveloppe sup. (approx.)', linewidth=2.5)
plt.plot(times, np.sqrt(allen_data), label='Enveloppe de Allen', color='tab:red', linewidth=2.5)
plt.plot(times, np.sqrt(baer_data), label='Enveloppe de Baer', color='tab:green', linewidth=2.5)
plt.plot(times, env_data2, label='Enveloppe supérieure', color='tab:blue', linewidth=2.5, alpha=1)

plt.xlabel('Temps [s]')
plt.ylabel('Amplitude [m/s]')
plt.xlim([t1, t2])
plt.ylim([-.2e-5, 1.0e-5])
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig('images/envelope_allen-baer.pgf', format='pgf')

In [97]:
t1, t2 = 46, 47
l = 1.5
plt.figure(figsize=(6,3))
ax = plt.subplot(111)
plt.plot(times, data, label='Signal', linewidth=5, color='grey')
plt.plot(times, abs_data, label='Valeur absolue', color='black', linewidth=l, linestyle='dotted')
plt.plot(times, env_data2, label='Enveloppe sup.', linewidth=2.5)
plt.plot(env_times, env_data, label='Enveloppe sup. (approx.)', linewidth=2.5)
plt.plot(times, np.sqrt(allen_data), label='Enveloppe de Allen',linewidth=2.5)
plt.plot(times, np.sqrt(baer_data), label='Enveloppe de Baer', linewidth=2.5)

plt.xlabel('Temps [s]')
plt.ylabel('Amplitude [m/s]')
plt.xlim([t1, t2])
plt.ylim([-.2e-5, 1.0e-5])
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=6)
plt.savefig('images/envelope_allen-baer.pgf', format='pgf')

## Détection par analyse de l'énergie

### Calcul de la fonction caractéristique

#### Méthode STA/LTA

In [98]:
# Paramètrage de la taille de la fenêtre
tlta = 20
tsta = 1

nsta, nlta = nstalta_from_times(times, tsta, tlta)

In [99]:
#fc_allen = stalta_allen(sqa_data, nsta, nlta)
fc_allen = stalta_allen_np(sqa_data, nsta, nlta)

#### Fonction caractéristique récursive d'Allen

In [100]:
fc_allen_recursiv = stalta_recursiv(sqa_data, nsta, nlta)

In [101]:
fig, axs = plt.subplots(2, figsize=(6,3))
ax1, ax2 = axs

t1, t2 = times[0], times[-1]
t1, t2 = 0, 100

color = 'tab:grey'
ax1.set_ylabel('Signal')
ax1.plot(times, data, color=color, linewidth=1.5)
ax1.set_xlim([t1, t2])
ax1.tick_params(axis='y')

ax2.set_ylabel('STA/LTA')
ax2.plot(times, fc_allen, color='tab:green', linewidth=1.5, label='Classique')
ax2.plot(times, fc_allen_recursiv, color='tab:blue', linewidth=1.5, label='Récursif')
ax2.set_xlim([t1, t2])
ax2.set_xlabel('Temps [s]')

plt.legend()
fig.tight_layout()
plt.savefig('images/stalta_classic-recursiv.pgf', format='pgf')

/tmp/ipykernel_101091/473557719.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(2, figsize=(6,3))


#### Variantes de Swindell et Snell et de Baer et Kradolfer

In [102]:
z = z_swindell_snell(sqa_data, nsta)

In [103]:
fc_baer = z_baer_kradolfer(baer_data, nsta)

In [104]:
fig, axs = plt.subplots(3, figsize=(6, 4), sharex=True)
ax1, ax2, ax3 = axs

t1, t2 = times[0], times[-1]
#t1, t2 = 40, 100

color = 'tab:grey'
ax1.set_ylabel('Signal')
ax1.plot(times, data, color=color, linewidth=1.5)
ax1.set_xlim([t1, t2])
ax1.tick_params(axis='y')

ax2.set_ylabel('Z-detect')
ax2.plot(times, z, color='tab:blue', linewidth=1.5)
#ax2.plot(times, z>0, color='tab:red', linewidth=1.5)
ax2.set_xlim([t1, t2])

ax3.set_ylabel('FC Baer')
ax3.plot(times, fc_baer, color='tab:blue', linewidth=1.5)
ax3.set_xlim([t1, t2])

ax3.set_xlabel('Temps [s]')

fig.tight_layout()
plt.savefig('images/fc-z.pgf', format='pgf')

### Pointage

#### Strasbourg

In [105]:
# # Téléchargement
# Séisme induit de Strasbourg
starttime = UTCDateTime(2021, 6, 26, 2, 58, 0)
duration = 5*60  # secondes
station = "ILLF" # station d'Illkirch
client = Client("RESIF")  # Réseau français
S = client.get_waveforms(network="FR", station="ILLF", location="00", channel="HH*", starttime=starttime, endtime=starttime+duration)
inventory = client.get_stations(network="FR", station=station, channel="HH*", starttime=starttime, level="response")
pre_process_data(S, inventory)

In [106]:
T = S.select(channel="HHZ")[0]
times = T.times(type='relative')
data = T.data

# Paramètrage de la taille de la fenêtre
tlta = 20
tsta = 1

sqa_data = data**2
fc_stewart = mdx_stewart(times, data, 10)
allen_data = allen_envelope_np(data)
baer_data = baer_envelope_np(times, data)

nsta, nlta = nstalta_from_times(times, tsta, tlta)

fc_allen = stalta_allen_np(allen_data, nsta, nlta)
fc_allen_recursiv = stalta_recursiv(allen_data, nsta, nlta)
z = z_swindell_snell(sqa_data, nsta)
fc_baer = z_baer_kradolfer(baer_data, nsta)

In [107]:
fig, axs = plt.subplots(5, figsize=(6,6), sharex=True)
ax1 = axs[0]
ax2 = axs[1]

t1, t2 = times[0], times[-1]
#t1, t2 = 30, 100
t1 = 50 # retirer le moment ou ST/LTA est nul

color = 'tab:grey'
ax1.set_ylabel('Signal')
ax1.plot(times, data, color=color, linewidth=1.5)
ax1.set_xlim([t1, t2])
ax1.tick_params(axis='y')

threshold = 5
potentials = potential_waves(times, z, threshold, tol=1, delta=3)
earthquake = any(potentials)

ax2.set_ylabel('Z-detect')
ax2.plot(times, z, color='tab:blue', linewidth=1.5)
ax2.hlines(y=threshold, xmin=t1, xmax=t2, linewidth=2, color='tab:green', ls='--', label=f'Seuil = {threshold}')
ax2.plot(times, (z>threshold)*-1, color='tab:green', linewidth=2, label=f'Dépassement')
if earthquake:
    earthquake_loc = potential_waves(times, z, threshold, tol=10, delta=3, before=30, after=30)
    ax2.plot(times, earthquake_loc*-2, color='tab:red', linewidth=3, label='Secousse')
else:
    ax2.legend("Absence de séisme", loc=1)
ax2.set_xlim([t1, t2])
ax2.legend(loc=1)

labels = ['STA/LTA', 'Récursif', 'Baer']
datasets = [fc_allen, fc_allen_recursiv, fc_baer]
thresholds = [5, 5, 200]
for k in range(len(labels)):
    threshold = thresholds[k]
    ax = axs[k+2]
    ax.set_ylabel(labels[k])
    ax.plot(times, datasets[k], color='tab:blue', linewidth=1.5)
    ax.set_xlim([t1, t2])
    ax.hlines(y=threshold, xmin=t1, xmax=t2, linewidth=2, color='tab:green', ls='--', label=f'Seuil = {threshold}')
    if earthquake:
        c = (max(datasets[k]) - min(datasets[k]))*.2
        potentials = potential_waves(times, datasets[k], threshold, tol=1)
        picks = point_potentials(datasets[k], potentials)
        picks = possible_picks(picks, earthquake_loc)
        print(f'Pointage avec {labels[k]}')
        if picks:
            ax.vlines(x=times[picks[0]], ymin=min(datasets[k])-c, ymax=max(datasets[k])+c, linewidth=2, color='tab:red', ls=':', label=f'$t_1$ = {times[picks[0]]:.2f} s')
            print(f'tP={(starttime + times[picks[0]])}')
        if len(picks) > 1:
            ax.vlines(x=times[picks[1]], ymin=min(datasets[k])-c, ymax=max(datasets[k])+c, linewidth=2, color='tab:orange', ls=':', label=f'$t_2$ = {times[picks[1]]:.2f} s')
            print(f'tS={(starttime + times[picks[1]])}')
    ax.legend(loc=1)

ax.set_xlabel('Temps [s]')
fig.tight_layout()
plt.savefig('images/pointage-strasbourg.pgf', format='pgf')

Pointage avec STA/LTA
tP=2021-06-26T03:00:44.370000Z
tS=2021-06-26T03:01:00.370000Z
Pointage avec Récursif
tP=2021-06-26T03:00:44.365000Z
tS=2021-06-26T03:01:00.335000Z
Pointage avec Baer
tP=2021-06-26T03:00:44.320000Z


#### Turquie

In [108]:
# # Téléchargement
# Séisme turquie
starttime = UTCDateTime(2023, 2, 6, 1, 22, 0)
duration = 5*60  # secondes
station = "ILLF" # station d'Illkirch
client = Client("RESIF")  # Réseau français
S = client.get_waveforms(network="FR", station="ILLF", location="00", channel="HH*", starttime=starttime, endtime=starttime+duration)
inventory = client.get_stations(network="FR", station=station, channel="HH*", starttime=starttime, level="response")
pre_process_data(S, inventory)

In [109]:
T = S.select(channel="HHZ")[0]
times = T.times(type='relative')
data = T.data

# Paramètrage de la taille de la fenêtre
tlta = 20
tsta = 1

sqa_data = data**2
fc_stewart = mdx_stewart(times, data, 10)
allen_data = allen_envelope_np(data)
baer_data = baer_envelope_np(times, data)

nsta, nlta = nstalta_from_times(times, tsta, tlta)

fc_allen = stalta_allen_np(allen_data, nsta, nlta)
fc_allen_recursiv = stalta_recursiv(allen_data, nsta, nlta)
z = z_swindell_snell(sqa_data, nsta)
fc_baer = z_baer_kradolfer(baer_data, nsta)

In [110]:
fig, axs = plt.subplots(5, figsize=(6,6), sharex=True)
ax1 = axs[0]
ax2 = axs[1]

t1, t2 = times[0], times[-1]
#t1, t2 = 30, 100

color = 'tab:grey'
ax1.set_ylabel('Signal')
ax1.plot(times, data, color=color, linewidth=1.5)
ax1.set_xlim([t1, t2])
ax1.tick_params(axis='y')

threshold = 5
potentials = potential_waves(times, z, threshold, tol=1, delta=3)
earthquake = any(potentials)

ax2.set_ylabel('Z-detect')
ax2.plot(times, z, color='tab:blue', linewidth=1.5)
ax2.hlines(y=threshold, xmin=t1, xmax=t2, linewidth=2, color='tab:green', ls='--', label=f'Seuil = {threshold}')
ax2.plot(times, (z>threshold)*-1, color='tab:green', linewidth=2, label=f'Dépassement')
if earthquake:
    earthquake_loc = potential_waves(times, z, 2, tol=10, delta=3, before=40, after=40)
    ax2.plot(times, earthquake_loc*-2, color='tab:red', linewidth=3, label='Secousse')
else:
    ax2.legend("Absence de séisme", loc=1)
ax2.set_xlim([t1, t2])
ax2.legend(loc=1)

labels = ['STA/LTA', 'Récursif', 'Baer']
datasets = [fc_allen, fc_allen_recursiv, fc_baer]
thresholds = [5, 5, 200]
for k in range(len(labels)):
    threshold = thresholds[k]
    ax = axs[k+2]
    ax.set_ylabel(labels[k])
    ax.plot(times, datasets[k], color='tab:blue', linewidth=1.5)
    ax.set_xlim([t1, t2])
    ax.hlines(y=threshold, xmin=t1, xmax=t2, linewidth=2, color='tab:green', ls='--', label=f'Seuil = {threshold}')
    if earthquake:
        c = (max(datasets[k]) - min(datasets[k]))*.2
        potentials = potential_waves(times, datasets[k], threshold, tol=3)
        picks = point_potentials(datasets[k], potentials)
        picks = possible_picks(picks, earthquake_loc)
        print(f'Pointage avec {labels[k]}')
        if picks:
            ax.vlines(x=times[picks[0]], ymin=min(datasets[k])-c, ymax=max(datasets[k])+c, linewidth=2, color='tab:red', ls=':', label=f'$t_1$ = {times[picks[0]]:.2f} s')
            print(f'tP={(starttime + times[picks[0]])}')
        if len(picks) > 1:
            ax.vlines(x=times[picks[1]], ymin=min(datasets[k])-c, ymax=max(datasets[k])+c, linewidth=2, color='tab:orange', ls=':', label=f'$t_2$ = {times[picks[1]]:.2f} s')
            print(f'tS={(starttime + times[picks[1]])}')
    ax.legend(loc=1)

ax.set_xlabel('Temps [s]')
fig.tight_layout()
plt.savefig('images/pointage-turquie.pgf', format='pgf')

Pointage avec STA/LTA
tP=2023-02-06T01:22:54.495000Z
tS=2023-02-06T01:24:44.065000Z
Pointage avec Récursif
tP=2023-02-06T01:22:56.485000Z
tS=2023-02-06T01:24:44.285000Z
Pointage avec Baer
